In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

import copy
import time
import sys
sys.path.insert(0, '~/CCPP/nudity_research/nudity-pytorch')
from data_ingestion import data_pipeline_pytorch_smaller_datasetq

In [5]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
if torch.cuda.is_available():
    model = model.cuda()
# Loss Function
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Using cache found in /Users/pyuvraj/.cache/torch/hub/pytorch_vision_v0.10.0


In [14]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
# set device to be cuda if available, otherwise it will be set to cpu
device = torch.device("cuda" if use_cuda else "cpu")

In [19]:
class RandomDataset(Dataset):
    def __init__(self, size, length, num_classes=2):
        self.len = length
        self.data = torch.randn(length, 3, size, size)
        self.labels = torch.randint(0, num_classes, (length,))
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    def __len__(self):
        return self.len
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 0}

In [20]:
# Generators
training_set = RandomDataset(224, 100)
training_generator = DataLoader(training_set, **params)
validation_set = RandomDataset(224, 20)
validation_generator = DataLoader(validation_set, **params)
dataloaders = {'train': training_generator, 'val': validation_generator}

In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
# Train and evaluate
dataset_sizes = {'train': len(training_set), 'val': len(validation_set)}
# Dummy dataset
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=2)



Epoch 0/1
----------
train Loss: 0.5941 Acc: 0.7300
val Loss: 0.6918 Acc: 0.5500

Epoch 1/1
----------
train Loss: 0.5211 Acc: 0.7200
val Loss: 0.6763 Acc: 0.5500

Training complete in 0m 16s
Best val Acc: 0.550000
